In [3]:
!python -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------------------- ---------------- 1.0/1.8 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 3.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1


In [4]:
pip install pandas mlxtend openpyxl

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [6]:
# Make sure Online Retail.xlsx is in the same directory
df = pd.read_excel("Online Retail.xlsx", engine='openpyxl')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:
# Remove rows with missing InvoiceNo or Description
df.dropna(subset=['InvoiceNo', 'Description'], inplace=True)

# Remove cancelled orders (InvoiceNo starts with 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Focus on one country (e.g., United Kingdom)
df = df[df['Country'] == 'United Kingdom']


In [9]:
# Create a basket matrix (products vs transactions)
basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')

# Convert quantities to 1/0 for presence/absence
def encode_units(x):
    return 1 if x >= 1 else 0

basket_sets = basket.apply(lambda col: col.map(encode_units))
basket_sets.head()

Description,20713,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
from mlxtend.frequent_patterns import apriori

# Convert item quantities to boolean values
def encode_units(x):
    return x >= 1  # Returns True or False

# Apply to the basket DataFrame
basket_sets = basket >= 1

# Generate frequent itemsets with minimum support = 2%
frequent_itemsets = apriori(basket_sets, min_support=0.02, use_colnames=True)

# Sort itemsets by support in descending order
frequent_itemsets = frequent_itemsets.sort_values(by="support", ascending=False)

# Display top itemsets
frequent_itemsets.head()

,support,itemsets
264,0.116027,(WHITE HANGING HEART T-LIGHT HOLDER)
101,0.103814,(JUMBO BAG RED RETROSPOT)
191,0.090261,(REGENCY CAKESTAND 3 TIER)
154,0.085387,(PARTY BUNTING)
126,0.074566,(LUNCH BAG RED RETROSPOT)


In [13]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.sort_values(by='lift', ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
52,(PINK REGENCY TEACUP AND SAUCER),"(GREEN REGENCY TEACUP AND SAUCER, ROSES REGENC...",0.037658,0.037551,0.026409,0.701280,18.675463,1.0,0.024995,3.221913,0.983490,0.541164,0.689625,0.702281
49,"(GREEN REGENCY TEACUP AND SAUCER, ROSES REGENC...",(PINK REGENCY TEACUP AND SAUCER),0.037551,0.037658,0.026409,0.703281,18.675463,1.0,0.024995,3.243278,0.983381,0.541164,0.691670,0.702281
51,(GREEN REGENCY TEACUP AND SAUCER),"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",0.050032,0.029248,0.026409,0.527837,18.047007,1.0,0.024945,2.055969,0.994338,0.499493,0.513611,0.715384
50,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.029248,0.050032,0.026409,0.902930,18.047007,1.0,0.024945,9.786461,0.973049,0.499493,0.897818,0.715384
53,(ROSES REGENCY TEACUP AND SAUCER ),"(GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY...",0.051264,0.030909,0.026409,0.515152,16.666982,1.0,0.024824,1.998751,0.990793,0.473583,0.499688,0.684785


In [14]:
# Filter rules with high confidence and lift
strong_rules = rules[(rules['lift'] > 3) & (rules['confidence'] > 0.5)]
strong_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

,antecedents,consequents,support,confidence,lift
1,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.042051,0.677308,6.524245
2,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.037551,0.750535,14.640537
3,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.037551,0.732497,14.640537
5,(JUMBO STORAGE BAG SUKI),(JUMBO BAG RED RETROSPOT),0.037390,0.617699,5.950055
7,(JUMBO SHOPPER VINTAGE RED PAISLEY),(JUMBO BAG RED RETROSPOT),0.035194,0.579876,5.585724
...,...,...,...,...,...
167,(CHARLOTTE BAG PINK POLKADOT),(CHARLOTTE BAG SUKI DESIGN),0.020570,0.569733,13.228575
170,(HOT WATER BOTTLE I AM SO POORLY),(CHOCOLATE HOT WATER BOTTLE),0.020302,0.596850,13.587809
178,(STRAWBERRY CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.020034,0.555721,15.189155
179,(WOODLAND CHARLOTTE BAG),(STRAWBERRY CHARLOTTE BAG),0.020034,0.547584,15.189155
